In [8]:
import pygame as p
%run ChessEngine.ipynb

WIDTH = HEIGHT = 512
CELLS = 8
SQ_SIZE = HEIGHT // CELLS
IMAGES = {}

In [9]:
def loadImages():
    pieces = ['bR','bN','bB','bQ','bK','bp','wR','wN','wB','wQ','wK','wp']
    for piece in pieces:
        IMAGES[piece] = p.image.load("images/" + piece + ".png")

In [10]:
def main():
    p.init()
    screen = p.display.set_mode((WIDTH, HEIGHT))
    running = True
    gs = GameState()
    # print(gs.board)
    loadImages()
    
    while running:
        for event in p.event.get():
            if event.type == p.QUIT:
                running = False
        p.display.flip() 
    p.quit()

In [ ]:
def drawGameState(screen, gs):
     

In [11]:
if __name__ == "__main__":
    main()

[['bR', 'bN', 'bB', 'bQ', 'bK', 'bB', 'bN', 'bR'], ['bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp', 'bp'], ['--', '--', '--', '--', '--', '--', '--', '--'], ['--', '--', '--', '--', '--', '--', '--', '--'], ['--', '--', '--', '--', '--', '--', '--', '--'], ['--', '--', '--', '--', '--', '--', '--', '--'], ['wp', 'wp', 'wp', 'wp', 'wp', 'wp', 'wp', 'wp'], ['wR', 'wN', 'wB', 'wQ', 'wK', 'wB', 'wN', 'wR']]
